In [9]:
import pandas as pd
import requests
import random
import time

# Set up header
HEADERS = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
  "Accept-Language": 'en-US,en;q=0.9',
  "Accept-Encoding": "gzip, deflate, br, zstd",
  "Referer": "https://tiki.vn/",
  "From": "",
  "af-ac-enc-dat": "",
  "x-api-source": "pc"
}


In [4]:
data = pd.read_csv('brand.csv')
data = pd.DataFrame(data)
data

,SubCategoryID,ProductID,BrandName
0,1795,184036446,Apple
1,1795,184059211,Apple
2,1795,57809866,Xiaomi
3,1795,120295859,Nokia
4,1795,123345348,Apple
...,...,...,...
23757,28926,249024285,OEM
23758,28926,204425773,OEM
23759,28926,13429081,OEM
23760,28926,72843893,OEM


In [6]:
apple = data[data['BrandName'] == 'Apple']
asus = data[data['BrandName'] == 'Asus']
hp = data[data['BrandName'] == 'HP']
samsung = data[data['BrandName'] == 'Samsung']

print('Number of Apple:', len(apple))
print('Number of Asus:', len(asus))
print('Number of HP:', len(hp))
print('Number of Samsung:', len(samsung))

Number of Apple: 15
Number of Asus: 210
Number of HP: 501
Number of Samsung: 311


In [21]:
def extract_product_data(product_ids_df):
    product_data_list = []

    for _, row in product_ids_df.iterrows():
        product_id = row['ProductID']
        URL = f"https://tiki.vn/api/v2/products/{product_id}"
        PARAMS = {}
        response = requests.get(URL, headers=HEADERS, params=PARAMS)
        time.sleep(random.uniform(3.2, 4.7))
        data = response.json()

        product_data = {
            'product_id': data['id'],
            'product_name': data.get('name', None),
            'product_url': data.get('short_url', None),
            'pricing_current': data.get('price', None),
            'pricing_original': data.get('original_price', None),
            'product_image_url': data.get('thumbnail_url', None),
            'inventory_status': data.get('inventory_status', None),
            'inventory_type': data.get('inventory_type', None),
            'created_date': data.get('day_ago_created', None),
            'quantity_sold': data.get('all_time_quantity_sold', None),
            'brand_id': data.get('brand', {}).get('id', None),
            'brand_name': data.get('brand', {}).get('name', None),
            'brand_slug': data.get('brand', {}).get('slug', None),
            'seller_id': data.get('current_seller', {}).get('id', 0) if data.get('current_seller') else 0,
            'seller_name': data.get('current_seller', {}).get('name', 0) if data.get('current_seller') else 0,
            'seller_link': data.get('current_seller', {}).get('link', 0) if data.get('current_seller') else 0,
            'seller_image_url': data.get('current_seller', {}).get('logo', 0) if data.get('current_seller') else 0,
            'category_id': data['categories']['id'] if 'categories' in data and data['categories'].get('is_leaf', False) else data['breadcrumbs'][-2]['category_id'] if 'breadcrumbs' in data and len(data['breadcrumbs']) >= 2 else None,
        }
        
        print(f"Fetched data for product {product_id}")

        product_data_list.append(product_data)

    print(f"Success fetching data for {len(product_data_list)} products")

    # Convert the product data list to a DataFrame
    product_data_df = pd.DataFrame(product_data_list)

    return product_data_df

In [22]:
# Extract product data for Samsung products
samsung_product_data = extract_product_data(samsung)
print(samsung_product_data)

Fetched data for product 197966909
Fetched data for product 213931247
Fetched data for product 247730209
Fetched data for product 244355503
Fetched data for product 273640492
Fetched data for product 245770363
Fetched data for product 262589811
Fetched data for product 255866251
Fetched data for product 253443894
Fetched data for product 270636882
Fetched data for product 249168949
Fetched data for product 249178196
Fetched data for product 274610468
Fetched data for product 274498287
Fetched data for product 274762212
Fetched data for product 273258830
Fetched data for product 207268158
Fetched data for product 212040689
Fetched data for product 248704404
Fetched data for product 273636845


In [16]:
samsung

,SubCategoryID,ProductID,BrandName
15,1795,197966909,Samsung
28,1795,213931247,Samsung
29,1795,247730209,Samsung
30,1795,244355503,Samsung
31,1795,273640492,Samsung
...,...,...,...
17011,12298,16396285,Samsung
17012,12298,124483882,Samsung
17013,12298,16613522,Samsung
17020,12298,72288260,Samsung
